In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np
from utils import show
from ipywidgets import interact
from IPython.display import clear_output
import json

import os, sys
sys.path.append(os.path.dirname(os.path.realpath("."))) # add parent path to import the pyfing package of this project

import pyfing as pf
from pyfing.segmentation import compute_segmentation_error

In [2]:
PATH_FVC = '../../datasets/'
PATH_GT = '../../datasets/segmentationbenchmark/groundtruth/'
PATH_RES = '../../results/'
PATH_PARAMS = '../parameters/segmentation/'

In [3]:
alg = pf.GradMag2SegmentationAlgorithm()
#alg = pf.DnnSegmentationAlgorithm(models_folder="../models/")
results = None
year = 2000
db = 1
subset = 'a'

In [4]:
@interact(y=[2000,2002,2004], n=[1,2,3,4])
def choose_db(y=year, n=db):
    global results, year, db
    year, db = y, n
    with open(f'{PATH_RES}fvc{year}_db{db}_{subset}_{type(alg).__name__}_res.txt', 'r') as filehandle:
        results = json.load(filehandle)

    #results = sorted(results, key=lambda r: r[2][0])
    alg.parameters =pf.GradMag2SegmentationParameters.load(f'{PATH_PARAMS}fvc{year}_db{db}_b_grad_mag2_params.json')

interactive(children=(Dropdown(description='y', options=(2000, 2002, 2004), value=2000), Dropdown(description=…

In [6]:
@interact(i=(0, len(results)-1))
def show_result(i=0):
    i, j, (err, dc) = results[i]
    img = cv.imread(f'{PATH_FVC}fvc{year}/db{db}_{subset}/{i}_{j}.png', cv.IMREAD_GRAYSCALE)
    gt = cv.bitwise_not(cv.imread(f'{PATH_GT}fvc{year}_db{db}_im_{i}_{j}seg.png', cv.IMREAD_GRAYSCALE))
    ir = []
    mask = alg.run(img, ir)
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    gt_contours, _ = cv.findContours(gt, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    
    img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    img_res = cv.drawContours(img_bgr.copy(), contours, -1, (255,0,0), 5)
    img_gt = cv.drawContours(img_bgr.copy(), gt_contours, -1, (0,192,0), 3)
    img_err = img_bgr.copy()
    false_positives = cv.bitwise_and(mask, cv.bitwise_not(gt))
    false_negatives = cv.bitwise_and(cv.bitwise_not(mask), gt)
    img_err[false_positives == 255,1:3] //= 2
    img_err[false_negatives == 255,0:3:2] //= 2
    cv.drawContours(img_err, gt_contours, -1, (0,192,0), 3)
    cv.drawContours(img_err, contours, -1, (255,0,0), 5)
    
    show(img, mask, (img_res, f'{i}_{j}'), (img_gt, 'Ground truth'), (img_err, f'Err = {err:.2f}% DC = {dc:.3f}'))
    show(*ir)

interactive(children=(IntSlider(value=0, description='i', max=799), Output()), _dom_classes=('widget-interact'…

In [8]:
pf.GradMagSegmentationAlgorithm.__name__

'GradMagSegmentationAlgorithm'

In [7]:
for y, db  in [(y, db) for y in (2000, 2002, 2004) for db in (1,2,3,4)]:
    p = pf.GradMag2SegmentationParameters.load(f'{PATH_PARAMS}fvc{y}_db{db}_b_grad_mag2_params.json')
    print(f"{y}\t{db}\t{p.sigma:.2f}\t{p.threshold:.2f}\t{p.percentile}\t{p.closing_count}\t{p.opening_count}\t{p.image_dpi}")

2000	1	3.00	0.18	95	2	15	500
2000	2	4.33	0.16	95	6	6	500
2000	3	4.33	0.22	95	6	12	500
2000	4	6.33	0.07	95	1	2	500
2002	1	3.67	0.03	95	4	9	500
2002	2	3.67	0.13	95	8	18	569
2002	3	5.00	0.25	95	4	9	500
2002	4	8.33	0.08	95	2	15	500
2004	1	2.33	0.02	95	8	18	500
2004	2	7.67	0.18	95	2	18	500
2004	3	5.00	0.12	95	4	15	512
2004	4	7.00	0.13	95	2	6	500
